# Comparison with the R version of `tximport`

This file is included to provide the code used to generate the output from `tximport` that the test in `test_correctness.py` compares against. It will note be automatically run by `pytest` and the assertions provided at the end are redundant, since they are already included in `test_correctness.py`.

In [1]:
!pip install rpy2==3.4.5 -q


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
!python3 -m rpy2.situation

rpy2 version:
3.4.5
Python version:
3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]
Looking for R's HOME:
    Environment variable R_HOME: None
    Calling `R RHOME`: /Library/Frameworks/R.framework/Resources
    Environment variable R_LIBS_USER: None
R's additions to LD_LIBRARY_PATH:

R version:
    In the PATH: R version 4.3.1 (2023-06-16) -- "Beagle Scouts"
    Loading R library from rpy2: OK
Additional directories to load R packages from:
None
C extension compilation:
  include:
  ['/Library/Frameworks/R.framework/Resources/include']
  libraries:
  ['pcre2-8', 'lzma', 'bz2', 'z', 'icucore', 'dl', 'm', 'iconv']
  library_dirs:
  ['/opt/R/arm64/lib', '/opt/R/arm64/lib']
  extra_compile_args:
  []
  extra_link_args:
  ['-F/Library/Frameworks/R.framework/..', '-framework', 'R']


In [3]:
%load_ext rpy2.ipython

In [4]:
%%R
R.version.string

[1] "R version 4.3.1 (2023-06-16)"


In [5]:
%%R
library(tximport)
library(readr)
dir <- "./data/fabry_disease"
tx2gene <- read_tsv(file.path(dir, "transcript_gene_mapping_human.tsv"))
rowMedians <- function(x) {
    apply(x, 1, median, na.rm = TRUE)
}
files <- c(
    file.path(dir, "SRR16504309_wt/quant.sf"),
    file.path(dir, "SRR16504310_wt/quant.sf"),
    file.path(dir, "SRR16504311_ko/quant.sf"),
    file.path(dir, "SRR16504312_ko/quant.sf")
)
countsFromAbundanceOptions <- c("no", "scaledTPM", "lengthScaledTPM")
for (idx in seq_along(countsFromAbundanceOptions)) {
    txi <- tximport(
        files,
        type = "salmon",
        tx2gene = tx2gene,
        countsFromAbundance = countsFromAbundanceOptions[idx],
        dropInfReps = TRUE,
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport.csv")
    if (!is.null(countsFromAbundanceOptions[idx])) {
        writePath <- gsub(".csv", paste0("_", countsFromAbundanceOptions[idx], ".csv"), writePath)
    }
    write.csv(txi$counts, writePath)
}
dataTypeOptions <- c("kallisto", "salmon")
for (idx in seq_along(dataTypeOptions)) {

    if (dataTypeOptions[idx] == "kallisto") {
        files <- c(
            file.path(dir, "SRR16504309_wt/abundance.h5"),
            file.path(dir, "SRR16504310_wt/abundance.h5"),
            file.path(dir, "SRR16504311_ko/abundance.h5"),
            file.path(dir, "SRR16504312_ko/abundance.h5")
        )
    } else {
        files <- c(
            file.path(dir, "SRR16504309_wt/quant.sf"),
            file.path(dir, "SRR16504310_wt/quant.sf"),
            file.path(dir, "SRR16504311_ko/quant.sf"),
            file.path(dir, "SRR16504312_ko/quant.sf")
        )
    }

    txi <- tximport(
        files,
        type = dataTypeOptions[idx],
        tx2gene = tx2gene,
        countsFromAbundance = "no",
        dropInfReps = FALSE,
        infRepStat = rowMedians,
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport_bootstrap.csv")
    writePath <- gsub(".csv", paste0("_", dataTypeOptions[idx], ".csv"), writePath)
    write.csv(txi$counts, writePath)
}
for (idx in seq_along(dataTypeOptions)) {

    if (dataTypeOptions[idx] == "kallisto") {
        files <- c(
            file.path(dir, "SRR16504309_wt/abundance.h5"),
            file.path(dir, "SRR16504310_wt/abundance.h5"),
            file.path(dir, "SRR16504311_ko/abundance.h5"),
            file.path(dir, "SRR16504312_ko/abundance.h5")
        )
    } else {
        files <- c(
            file.path(dir, "SRR16504309_wt/quant.sf"),
            file.path(dir, "SRR16504310_wt/quant.sf"),
            file.path(dir, "SRR16504311_ko/quant.sf"),
            file.path(dir, "SRR16504312_ko/quant.sf")
        )
    }

    txi <- tximport(
        files,
        type = dataTypeOptions[idx],
        tx2gene = tx2gene,
        countsFromAbundance = "no",
        txOut = TRUE,
        dropInfReps = FALSE,
        infRepStat = rowMedians,
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport_bootstrap_transcripts.csv")
    writePath <- gsub(".csv", paste0("_", dataTypeOptions[idx], ".csv"), writePath)
    write.csv(txi$counts, writePath)
}

Rows: 244191 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): transcript_id, gene_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 31380

R[write to console]: summarizing abundance

R[write to console]: summarizing counts

R[write to console]: summarizing length

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 31380

R[write to console]: summarizing abundance

R[write to console]: summarizing counts

R[write to console]: summarizing length

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 2 
R[write to console]: 3 
R[write to console]: 4 
R[write to console]: 

R[write to console]: transcripts missing from tx2gene: 31380

R[write to console]: summarizing abundance

R[

In [6]:
!pytximport -i ./data/fabry_disease/SRR16504309_wt/quant.sf -i ./data/fabry_disease/SRR16504310_wt/quant.sf -i ./data/fabry_disease/SRR16504311_ko/quant.sf -i ./data/fabry_disease/SRR16504312_ko/quant.sf -t salmon -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -ow -o ./data/fabry_disease/counts_pytximport_no.csv
!pytximport -i ./data/fabry_disease/SRR16504309_wt/quant.sf -i ./data/fabry_disease/SRR16504310_wt/quant.sf -i ./data/fabry_disease/SRR16504311_ko/quant.sf -i ./data/fabry_disease/SRR16504312_ko/quant.sf -t salmon -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -ow -o ./data/fabry_disease/counts_pytximport_scaledTPM.csv -c scaled_tpm
!pytximport -i ./data/fabry_disease/SRR16504309_wt/quant.sf -i ./data/fabry_disease/SRR16504310_wt/quant.sf -i ./data/fabry_disease/SRR16504311_ko/quant.sf -i ./data/fabry_disease/SRR16504312_ko/quant.sf -t salmon -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -ow -o ./data/fabry_disease/counts_pytximport_lengthScaledTPM.csv -c length_scaled_tpm

2024-07-29 16:12:29,433: Starting the import.
Reading quantification files: 4it [00:01,  2.36it/s]
2024-07-29 16:12:31,270: Converting transcript-level expression to gene-level expression.
2024-07-29 16:12:31,770: Not all transcripts are present in the mapping. 31380 out of 253181 missing. Removing the missing transcripts.
2024-07-29 16:12:32,091: Matching gene_ids.
2024-07-29 16:12:32,294: Creating gene abundance.
2024-07-29 16:12:32,612: Creating gene counts.
2024-07-29 16:12:32,613: Creating lengths.
2024-07-29 16:12:32,753: Replacing missing lengths.
2024-07-29 16:12:40,884: Creating gene expression dataset.
2024-07-29 16:12:40,923: Saving the gene-level expression to: data/fabry_disease/counts_pytximport_no.csv.
2024-07-29 16:12:41,014: Finished the import in 11.58 seconds.
2024-07-29 16:12:42,764: Starting the import.
Reading quantification files: 4it [00:01,  2.24it/s]
2024-07-29 16:12:44,690: Converting transcript-level expression to gene-level expression.
2024-07-29 16:12:45,1

In [7]:
import pandas as pd

counts_tximport_no = pd.read_csv("./data/fabry_disease/counts_tximport_no.csv")
counts_tximport_scaledTPM = pd.read_csv("./data/fabry_disease/counts_tximport_scaledTPM.csv")
counts_tximport_lengthScaledTPM = pd.read_csv("./data/fabry_disease/counts_tximport_lengthScaledTPM.csv")

counts_pytximport_no = pd.read_csv("./data/fabry_disease/counts_pytximport_no.csv")
counts_pytximport_scaledTPM = pd.read_csv("./data/fabry_disease/counts_pytximport_scaledTPM.csv")
counts_pytximport_lengthScaledTPM = pd.read_csv("./data/fabry_disease/counts_pytximport_lengthScaledTPM.csv")
counts_pytximport_no.columns = counts_tximport_no.columns
counts_pytximport_scaledTPM.columns = counts_tximport_scaledTPM.columns
counts_pytximport_lengthScaledTPM.columns = counts_tximport_lengthScaledTPM.columns

pd.testing.assert_frame_equal(counts_tximport_no, counts_pytximport_no)
pd.testing.assert_frame_equal(counts_tximport_scaledTPM, counts_pytximport_scaledTPM)
pd.testing.assert_frame_equal(counts_tximport_lengthScaledTPM, counts_pytximport_lengthScaledTPM)

## Compare outputs for transcript-level summarization

In [8]:
%%R
dir <- "./data/salmon"
files_protein_coding <- c(
  file.path(dir, "quant.sf")
)
tx2gene <- read_tsv(file.path("./data/fabry_disease", "transcript_gene_mapping_human.tsv"))
countsFromAbundanceOptions <- c("scaledTPM", "dtuScaledTPM")
for (idx in seq_along(countsFromAbundanceOptions)) {
    txi <- tximport(
        files_protein_coding,
        type = "salmon",
        tx2gene = tx2gene,
        txOut = TRUE,
        countsFromAbundance = countsFromAbundanceOptions[idx],
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport.csv")
    if (!is.null(countsFromAbundanceOptions[idx])) {
        writePath <- gsub(".csv", paste0("_", countsFromAbundanceOptions[idx], ".csv"), writePath)
    }
    write.csv(txi$counts, writePath)
}

Rows: 244191 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): transcript_id, gene_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 

R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 



In [9]:
!pytximport -i ./data/salmon/quant.sf -m ./data/fabry_disease/transcript_gene_mapping_human.tsv -o ./data/salmon/counts_pytximport_dtuScaledTPM.csv -t salmon -tx -c dtu_scaled_tpm

2024-07-29 16:13:11,427: Starting the import.
Reading quantification files: 1it [00:00, 280.09it/s]
2024-07-29 16:13:11,887: Setting counts to length scaled TPM.
Traceback (most recent call last):
  File "/Users/au734063/Documents/code/pytximport-publish/pytximport/.venv/bin/pytximport", line 8, in <module>
    sys.exit(cli())
  File "/Users/au734063/Documents/code/pytximport-publish/pytximport/.venv/lib/python3.10/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/Users/au734063/Documents/code/pytximport-publish/pytximport/.venv/lib/python3.10/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/Users/au734063/Documents/code/pytximport-publish/pytximport/.venv/lib/python3.10/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/Users/au734063/Documents/code/pytximport-publish/pytximport/.venv/lib/python3.10/site-packages/click/core.py", line 783, in 

In [10]:
counts_tximport_dtuScaledTPM = pd.read_csv("./data/salmon/counts_tximport_dtuScaledTPM.csv", index_col=0).sort_index()
counts_pytximport_dtuScaledTPM = pd.read_csv(
    "./data/salmon/counts_pytximport_dtuScaledTPM.csv", index_col=0
).sort_index()
# cut the transcript version from the index
counts_tximport_dtuScaledTPM.index = counts_tximport_dtuScaledTPM.index.str.split(".").str[0]
counts_pytximport_dtuScaledTPM.columns = counts_tximport_dtuScaledTPM.columns

pd.testing.assert_frame_equal(counts_tximport_dtuScaledTPM, counts_pytximport_dtuScaledTPM)

## Compare outputs for RSEM

In [11]:
%%R
dir <- "./data/rsem"
files_protein_coding <- c(
  file.path(dir, "test.genes.results.gz")
)
tx2gene <- read_tsv(file.path("./data/fabry_disease", "transcript_gene_mapping_human.tsv"))
countsFromAbundanceOptions <- c("no")
for (idx in seq_along(countsFromAbundanceOptions)) {
    txi <- tximport(
        files_protein_coding,
        type = "rsem",
        tx2gene = tx2gene,
        txIn = FALSE,
        countsFromAbundance = countsFromAbundanceOptions[idx],
        ignoreTxVersion = TRUE,
        ignoreAfterBar = TRUE
    )
    writePath <- file.path(dir, "counts_tximport.csv")
    if (!is.null(countsFromAbundanceOptions[idx])) {
        writePath <- gsub(".csv", paste0("_", countsFromAbundanceOptions[idx], ".csv"), writePath)
    }
    write.csv(txi$counts, writePath)
}

Rows: 244191 Columns: 2
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (2): transcript_id, gene_id

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


R[write to console]: reading in files with read_tsv

R[write to console]: 1 
R[write to console]: 

